In [ ]:
# Relevant libraries
import pandas as pd
import scipy.optimize as sc
import scipy.stats as st
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from scipy.stats import f
import statsmodels.api as sm
from scipy.optimize import OptimizeWarning
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import multiprocessing
from joblib import Parallel, delayed
from tqdm import tqdm
import warnings


In [ ]:
#######
# Creates several plots to create a color gradient of the funk equation ppm prediction
#######

plt.figure(figsize=(10,8))
# Create a scatter plot, with the color determined by y_pred

resistance_range = np.linspace(0, 50, 1000)  # 10000 points between 0 and 1000
rh_range = np.linspace(5, 100, 1000)  # 10000 points between 0 and 100

# Create a grid of Resistance and RelativeHumidity values
resistance_values, rh_values = np.meshgrid(resistance_range, rh_range)


# Compute predicted PPM for all combinations of Resistance and RelativeHumidity
pred_ppm = funkEQ((resistance_values, rh_values), *popt)

plt.figure(figsize=(10, 8))

# Generate a large number of levels for smoother color transitions
num_levels = 20
plt.contourf(resistance_values, rh_values, pred_ppm, num_levels, cmap='viridis')

plt.colorbar(label='Predicted PPM')
plt.xlabel('Resistance Ratio')
plt.ylabel('Relative Humidity')
plt.title('Color Gradient Graph for Predicted PPM')
plt.show()
# Create a figure
plt.figure(figsize=(10,8))

# Get min and max values of y_pred and pred_ppm
vmin = min(y_pred.min(), pred_ppm.min())
vmax = max(y_pred.max(), pred_ppm.max())

# Draw the scatter plot with vmin and vmax
sc = plt.scatter(xDataResistance, xDataRH, c=y_pred, cmap='viridis', vmin=vmin, vmax=vmax)

# Compute predicted PPM for all combinations of Resistance and RelativeHumidity
pred_ppm = funkEQ((resistance_values, rh_values), *popt)

# Draw the contour plot with vmin and vmax
plt.contourf(resistance_values, rh_values, pred_ppm, num_levels, cmap='viridis', alpha=0.4, vmin=vmin, vmax=vmax)

# Create a colorbar
plt.colorbar(sc, label='Predicted PPM')

# Label axes and title
plt.xlabel('Resistance Ratio')
plt.ylabel('Relative Humidity')
plt.title('Color Gradient Graph for Predicted PPM')

# Show the plot
plt.show()

plt.figure(figsize=(10, 8))
sc = plt.scatter(xDataResistance, xDataRH, c=y_pred, cmap='viridis', vmin=vmin, vmax=vmax)
# Create a colorbar
plt.colorbar(sc, label='Predicted PPM')
plt.xlabel('Resistance Ratio')
plt.ylabel('RelativeHumidity')
plt.title('Color gradient graph')
plt.show()


In [ ]:
######
# Does basically the same thing as the cell above
######

# Create a grid of Resistance and RelativeHumidity values
resistance_values, rh_values = np.meshgrid(resistance_range, rh_range)


# Compute predicted PPM for all combinations of Resistance and RelativeHumidity
pred_ppm = funkEQ((resistance_values, rh_values), *popt)

plt.figure(figsize=(10, 8))

plt.imshow(pred_ppm, origin='lower', extent=[resistance_values.min(), resistance_values.max(), rh_values.min(), rh_values.max()], aspect='auto', cmap='viridis')

plt.colorbar(label='Predicted PPM')
plt.xlabel('Resistance')
plt.ylabel('Relative Humidity')
plt.title('Color Gradient Graph for Predicted PPM')
plt.show()
# Create a figure
plt.figure(figsize=(10, 8))

# Create a smooth color gradient with imshow() and set its opacity to 50%
plt.imshow(pred_ppm, origin='lower', extent=[resistance_values.min(), resistance_values.max(), rh_values.min(), rh_values.max()], aspect='auto', cmap='viridis', alpha=0.6)

# Add a scatter plot on top of the color gradient with 100% opacity
sc = plt.scatter(xDataResistance, xDataRH, c=y_pred, cmap='viridis', alpha=1.0)

plt.colorbar(label='Predicted PPM')
plt.xlabel('Resistance')
plt.ylabel('Relative Humidity')
plt.title('Color Gradient Graph for Predicted PPM')

plt.show()

In [ ]:
#######
# function to view predicted ppm across temperatures while holding ppm and humidity constant
#######
import matplotlib.cm as cm

file2 = pd.read_csv("/Users/benfunk/DataspellProjects/MethaneDataScience/Output/BenSummaryJustMatrix.csv")
file2=file2[file2["Target PPM"]<=0]
file2=file2[file2["RelativeHumidity"]<=50]
file2=file2[file2["RelativeHumidity"]>=40]
xDataResistance = file2.loc[:, 'Resistance']
xDataRatio = file2.loc[:, 'Ratio']
xDataRH = file2.loc[:, 'RelativeHumidity']
xDataTemp = file2.loc[:, 'Temperature']
yDataTargetPPM = file2.loc[:, 'Target PPM']

df = file2
y_pred = funkEQ((xDataResistance, xDataRH), *popt)
df['Predicted PPM'] = y_pred
# Create a scatter plot with colorbar
sc = plt.scatter(df.loc[:, "Temperature"],y_pred, c=xDataResistance, cmap=cm.viridis)
plt.colorbar(sc, label='Resistance Ratio')
plt.xlabel("Temperature")
plt.ylabel('Predicted PPM')

# Calculate mean prediction for each temperature and plot it
mean_pred_per_temp = df.groupby('Temperature')['Predicted PPM'].mean()
plt.plot(mean_pred_per_temp.index, mean_pred_per_temp.values, color='r', label='Mean prediction per temperature')
plt.legend()
plt.title('Temperature at 0 PPM')

plt.show()